In [1]:
from ecg_qc import ecg_qc, sqi_computing
import os, sys
sys.path.append('../')
from dags.tasks.detect_qrs import read_mit_bih_noise, sampling_frequency
import plotly.express as px
import pandas as pd


In [2]:
def apply_algo(algo, signal, length_chunk, sampling_frequency):

    # Preprocess signal : chunks of length_chunk seconds
    n = length_chunk * sampling_frequency
    signal_subdiv = [signal[i * n:(i + 1) * n]
                        for i in range((len(signal) + n - 1) // n)]
    # Padding on last chunk if necessary
    m = len(signal_subdiv[-1])
    if m < n:
        signal_subdiv[-1] += [0 for j in range(n - m)]
    # Apply ecg_qc on each chunk
    signal_sqis = [algo.compute_sqi_scores(x) for x in signal_subdiv]
    signal_quality = [algo.get_signal_quality(x) for x in signal_subdiv]

    return signal_sqis, signal_quality

In [3]:
rfc = '../models/rfc.joblib'
model = '../models/model.pkl'
model_mieux = '../models/model_mieux.pkl'
models = [rfc, model, model_mieux]

data_path = '../data'
SNR = 'e00'

data_generator = read_mit_bih_noise(SNR, data_path)
pat118 = next(data_generator)
pat119 = next(data_generator)
signal_118_MLII = list(pat118[1]['MLII'])
signal_118_V1 = list(pat118[1]['V1'])
signal_119_MLII = list(pat119[1]['MLII'])
signal_119_V1 = list(pat119[1]['V1'])

In [4]:
portion_118_MLII = signal_118_MLII[:21601] # 1 min = 21600 pts 
portion_118_V1 = signal_118_V1[:21601]
portion_119_MLII = signal_119_MLII[:21601]
portion_119_V1 = signal_119_V1[:21601]

# fig = px.line(y=portion_118_MLII, title='118 MLII') 
# fig.show()
# fig2 = px.line(y=portion_118_V1, title='118 V1') 
# fig2.show()
# fig3 = px.line(y=portion_119_MLII, title='119 MLII') 
# fig3.show()
# fig4 = px.line(y=portion_119_V1, title='119 V1')
# fig4.show()

In [5]:
length_chunk = 9 # seconds
signals = [portion_118_MLII, portion_118_V1, portion_119_MLII, portion_119_V1]
sig_names = ['118 MLII', '118 V1', '119 MLII', '119 V1']
df = pd.DataFrame(columns=['model', 'signal', 'segment', 'qsqi', 'csqi', 'ssqi', 'ksqi', 'psqi', 'bassqi', 'quality'])
for model in models :
    algo = ecg_qc(model=model, sampling_frequency=sampling_frequency)
    model_name = model.split('/')[-1].split('.')[0]
    for i in range(len(signals)):
        sig_name = sig_names[i]
        sig = signals[i]
        sqis, qual = apply_algo(algo, sig, length_chunk, sampling_frequency)
        for j in range(len(qual)):
            data = [model_name, sig_name, j] + sqis[j][0] + [qual[j]]
            df = df.append(
                pd.DataFrame(
                    [data], columns=['model', 'signal', 'segment', 'qsqi', 'csqi', 'ssqi', 'ksqi', 'psqi', 'bassqi', 'quality']
                ), 
                ignore_index=True)

print(df)


/home/nacm/github/test_ecg_qc/venv/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.2 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/home/nacm/github/test_ecg_qc/venv/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.2 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/home/nacm/github/test_ecg_qc/venv/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator GridSearchCV from version 0.24.2 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/home/nacm/github/test_ecg_qc/venv/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifi

          model    signal segment  qsqi  csqi  ssqi   ksqi  psqi  bassqi  \
0           rfc  118 MLII       0  0.95  0.63 -0.28   4.05  0.64    0.31   
1           rfc  118 MLII       1  1.00  0.61 -0.25   2.58  0.63    0.30   
2           rfc  118 MLII       2  0.91  0.60 -0.03   3.51  0.66    0.31   
3           rfc  118 MLII       3  0.91  0.60 -0.12   4.20  0.63    0.35   
4           rfc  118 MLII       4  0.87  0.57 -0.21   2.63  0.65    0.34   
..          ...       ...     ...   ...   ...   ...    ...   ...     ...   
79  model_mieux    119 V1       2  0.95  0.66 -2.09  11.47  0.67    0.28   
80  model_mieux    119 V1       3  0.84  0.64 -2.01  11.41  0.68    0.29   
81  model_mieux    119 V1       4  0.84  0.64 -1.86   9.19  0.68    0.28   
82  model_mieux    119 V1       5  0.90  0.61 -2.08  11.74  0.68    0.27   
83  model_mieux    119 V1       6  0.92  0.57  0.66  -1.48  0.56    0.40   

   quality  
0        1  
1        1  
2        0  
3        0  
4        0  
..     ..